#TensorFlow Mecanica 101

El objetivo de este tutorial es mostrar como utilizar TensorFlow para entrenar y evaluar una red neuronal feed-forward para la clasificacion de digitos escritos a mano mediante el dataset de MNIST. Se revisan dos archivos de python y se examinan las funciones principales de dichos archivos.

#Archivos usados en el Tutorial

En este tutorial se usan los siguientes dos archivos los cuales se pueden encontrar en el github del proyecto TensorFlow los cuales son:

* mnist.py (El codigo para construir un modelo MNIST plenamente conectado.)
* fully_connected_feed.py (El codigo principal para entrenar el modelo MNIST construido contra el conjunto de datos descargados usando un diccionario de alimentacion.)

Si se ejecuta el archivo fully_connected_feed.py se empieza a entrenar la red neuronal.

#Preparar los Datos

MNIST es un problema clasico de machine learning el cual consiste en un conjunto de imagenes de tamanio de 28 x 28 con digitos escritos a mano de 0-9 en lo cual se busca clasificar las imagenes de acuerdo a la clase a la que pertenecen por ejemplo si en la imagen hay un 0 se espera que se clasifique dicha imagen en la clase 0.

#Descarga

En el archivo fully_connected_feed.py hay una funcion que se llama run_training() en dicha funcion hay una linea de codigo que dice input_data.read_data_sets() la cual consiste en descargar los datos en una carpeta y regresa un diccionario el cual se llama data_sets en donde se encuentran las instancias a dichos datos.

data_sets = input_data.read_data_sets(FLAGS.train_dir, FLAGS.fake_data)

Los datos a tratar son los siguientes:

* data_sets.train (Contiene 55000 imagenes y etiquetas para entrenar)
* data_sets.validation (Contiene 5000 imagenes y etiquetas para validar la exactitud del entrenamiento)
* data_sets.test (Contiene 10000 imagenes y etiquetas para testear la exactitud del entrenamiento)


#Inputs y Placeholders

En el archivo fully_connected_feed.py hay una funcion que se llama placeholder_inputs() la cual crea dos ops tf.placeholders las cuales definen el tamanio de las entradas (inputs) incluyendo el tamanio del batch (batch_size) para el uso en el grafo en el cual se entrenaran los ejemplos y se les dara fed.

images_placeholder = tf.placeholder(tf.float32, shape=(batch_size, IMAGE_PIXELS))

labels_placeholder = tf.placeholder(tf.int32, shape=(batch_size))

Cuando se ejecute el loop de entrenamiento la imagen completa y las etiquetas del dataset se ajustan al tamanio del batch en cada iteracion ademas se emparejaran con las ops de placeholders y se pasaran dentro de una funcion de sesion tal como sess.run() esta funcion tendra como parametro a feed_dict.

#Construyendo el Grafo

Despues de crear los placegolders para los datos el grafo se construye a partir del archivo mnits.py de acuerdo a un patron de tres etapas las cuales son inference(), loss(), y training().

1. inference() - Construye el grafo en la medida que es requerido para ejecutar la red y hacer predicciones
2. loss() - Agrega las ops requeridas al grafo de inferencia para generar perdida.
3. training() - Agrega a el grafo de perdida las ops requeridas para calcular y aplicar los gradientes.

#Inferencia

La funcion de inference() construye el grafo en la medida que es requerido para regresar un tensor que contiene las salidas de las predicciones.

Toma las imagenes del placeholder como entrada y se basa en la parte superior de un par de capas completamente conectadas con la activacion Relu seguido de la capa lineal del nodo diez especificando la salida de logits (Inversa de la funcion logistica).

Cada capa se crea debajo de un unico tf.name_scope que actua como un prefijo a los objetos creados dendro de ese scope.

with tf.name_scope('hidden1') as scope:

Dentro del scope definido los pesos y bias que se usaran por cada una de estas capas son generados dentro de instancias a tf.Variable con los tamanios deseados.

weights = tf.Variable(tf.truncated_normal([IMAGE_PIXELS, hidden1_units], stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))), name='weights')

biases = tf.Variable(tf.zeros([hidden1_units]), name='biases')

Para una instancia que se crea debajo de el scope de hidden1 el unico nombre que se le da a la variable pesos debe ser "hidden1/weights".

Cada variable se le da un inicializador de ops como parte de su construccion.

En los casos mas comunes los pesos son inicializados con la funcion tf.truncated_normal y se le da el tamanio del tensor 2-d la primera dimension representa el numero de unidades en la capa desde la cual los pesos estan conectados y la segunda dimension representa el numero de unidades en la capa hacia los pesos conectados. 

Para la primer capa con el nombre hidden1 las dimensiones son [IMAGE_PIXELS, hidden1_units] porque los pesos estan conectados a la entrada de la imagen de la capa hidden1. La funcion tf.truncated_normal genera e inicializa una distribucion aleatoria dada la media y la desviacion estandar.

Luego los biases son inicializados con tf.zeros para asegurarse que todos comienzan con valores de 0  y su tamanio es simplemente el numero de unidades en la capa en la cual estan conectados.

El grafo tiene tres ops primarias: dos ops tf.nn.relu de wrapping y tf.matmul para las capas ocultas ademas de una extra tf.matmul para los logits son creadas en cada turno con instancias separadas de tf.Variable conectadas a cada entrada de los placeholders o la salida de los tensores de la capa anterior.

hidden1 = tf.nn.relu(tf.matmul(images, weights) + biases)

hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)

logits = tf.matmul(hidden2, weights) + biases

Finalmente el tensor de logits contendra la salida la cual se regresara.

#Perdida

La funcion loss() construye el grafo agregando las loss ops requeridas.

Primero los valores de los labels_placeholder estan codificados en tensores de 1-hot values. por ejemplo, si la clase identifica el valor '3' este se convertira en:

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

batch_size = tf.size(labels)
labels = tf.expand_dims(labels, 1)
indices = tf.expand_dims(tf.range(0, batch_size, 1), 1)
concated = tf.concat(1, [indices, labels])
onehot_labels = tf.sparse_to_dense(concated, tf.pack([batch_size, NUM_CLASSES]), 1.0, 0.0)
    
La op tf.nn.softmax_cross_entropy_with_logits se agrega para comparar la salida de logits de la funcion inference() y las etiquetas 1-hot

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, onehot_labels, name='xentropy')

Si luego se usa tf.reduce_mean para obtener el promedio de cross-entropy a travez de la dimension del batch (La primera dimension) es la perdida total.

loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')

Despues el tensor contendra el valor de perdida el cual se regresara.

Nota: La idea de cross-entropy viene de el campo de la teoria de la informacion que describe que tan mal son las predicciones de la red neuronal dado lo que actualmente es correcto. 

#Entrenamiento

La funcion training() agrega las operaciones necesarias para minimizar la perdida por medio de descenso de gradiente.

Primero toma el tensor loss de la funcion loss() y se lo entrega a un tf.scalar_summary una op para generar una recopilacion  de valores dentro del archivo de eventos para cuando lo use SummaryWriter una funcion que se vera mas abajo. En este caso se emitira un valor instantaneo de la perdida cada vez que la recopilacion se escriba fuera.

tf.scalar_summary(loss.op.name, loss)

Lo siguiente es instanciar la funcion tf.train.GradientDescentOptimizer la cual es la responsable de aplicar los gradientes con lo requerido por la taza de aprendisaje.

optimizer = tf.train.GradientDescentOptimizer(FLAGS.learning_rate)

Se genera una variable que contiene un contador para los pasos del entrenamiento global y la op minimize() tiene dos usos los cuales som  actualizar los pesos de entrenamiento en el sistema e incrementar los pasos globales. Esto se conoce por convencion como train_op y la debe ejecutar una sesion de TensorFlow con el fin de inducir un paso completo.

global_step = tf.Variable(0, name='global_step', trainable=False)

train_op = optimizer.minimize(loss, global_step=global_step)

El tensor contiene la salida de la op training y se regresa.



#Entrenando el Modelo

Una vez que el grafo es construido se puede entrenar iterativamente y evaluar en el loop con el codigo del usuario en el archivo fully_connected_feed.py.

#El Grafo

Al principio de la funcion run_training() hay un comando en python que indica que toda la construccion de las ops son asociadas con la instancia global por default tf.Graph 

with tf.Graph().as_default():

Un tf.Graph es una coleccion de ops que pueden ser ejecutadas a la vez como un grupo. Muchos de los usos de TensorFlow usaran solo lo necesario para confiar en el grafo por default.

#La Sesion

Una vez que la preparacion se ha completado y todas las ops necesarias se han generado se pasa a crear un tf.Session para recorrer el grafo.

Alternativamente una sesion se puede generar dendro de un bloque para el scoping:

with tf.Session() as sess:

Si no se manda un parametro a la sesion se indica qie el codigo se adjuntara (o se creara si no se a creado) una sesion local por default.

Inmediatamente despues de crear la sesion todas las instancias de tf.Variable se inicializaran llamando a sess.run() con su op de inicializacion.

init = tf.initialize_all_variables()
sess.run(init)

La funcion sess.run() ejecutara un subconjunto del grafo que corresponde a las ops que se pasaron como parametros. En la primera llamda la init de la op es un tf.group que contiene solo los inicializadores para las variables. Ninguna otra parte del grafo se ejecuta aqui eso se ejecuta en el loop de entrenamiento el cual se vera enseguida.

#Loop de Entrenamiento

Despues de inicializar las variables con la sesion se comienza a entrenar.

El codigo del usuario controla el entrenamiento por pasos un simple loop es util para el entrenamiento.

for step in xrange(max_steps):
    sess.run(train_op)

#Alimentar el Grafo

Por cada paso el codigo generara un feed dictionary el cual contendra un conjunto de ejemplos sobre el cual se entrena en el paso tecleado por las ops del placeholder.

En la funcion fill_feed_dict() el dataset dado es consultado para el siguiente batch_size el cual es un conjunto de imagenes y etiquetas y tensores emparejados con los placeholders se llenan con las siguientes imagenes y etiquetas.

images_feed, labels_feed = data_set.next_batch(FLAGS.batch_size)

Un objeto diccionario en python es generado con los placeholders como llaves y las representaciones de los tensores feed como los valores.

feed_dict = {
    images_placeholder: images_feed,
    labels_placeholder: labels_feed,
}

Esto se pasa dentro de la funcion sess.run() el parametro feed_dict provee la entrada de los ejemplos para este paso de entrenamiento.

#Comprobando el estado

El codigo especifica dos valores a fetch en su llamada de ejecucion: [train_op, loss].

for step in xrange(FLAGS.max_steps):

    feed_dict = fill_feed_dict(data_sets.train, images_placeholder, labels_placeholder)
    
    _, loss_value = sess.run([train_op, loss], feed_dict=feed_dict)

Porque hay dos valores pra fetch, la funcion sess.run() regresa una tupla con dos objetos. Cada tensor en la lista de valores de fetch corresponde a un arreglo de numpy el la tupla que se regresa, se llena con el valor de ese tensor durante este paso de entrenamiento. Desde train_op es una operacion con ningun valor de salida, el elemento correspondiente en la tupla que se regresa es None y ademas se descarta. De cualquier manera el valor del tensor loss sera NaN si el modelo diverge durante el entrenamiento entonces se captura este valor para logging.

Asumiendo que se ejecuta el entrenamiento correctamente sin Nans el loop de entrenamiento tambien imprime el estado cada 100 pasos para hacer al usuario saber el estado de entrenamiento.

if step % 100 == 0:
    
    print 'Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration)


#Visualizar el Estado

En el orden de emitir los archivos de los eventos usados por el TensorBoard todas las recopilaciones (En este caso solo una) se colectan dentro de una sola op durante la fase de construccion.

summary_op = tf.merge_all_summaries()

Despues de que la sesion es creada un tf.train.SummaryWriter puede ser instanciado para escribir los archivos de eventos los cuales contienen el grafo en si y los valores de las recopilaciones.

summary_writer = tf.train.SummaryWriter(FLAGS.train_dir, graph_def=sess.graph_def)

Finalmente los archivos de los eventos se actualizaran con los nuevos valores del recopilatoria cada vez que summary_op se ejecute y la salida sea pasada a la funcion escritor add_summary().

summary_str = sess.run(summary_op, feed_dict=feed_dict)
summary_writer.add_summary(summary_str, step)

Cuando los archivos de eventos se han escrito TensorBoard puede ejecutarse contra la carpeta de entrenamiento para mostrar los valores de los recopilatorios.

#Guardar un Checkpoint

EN orden de emitir un archivo Checkpoint se puede usar despues pra restaurar el modelo para adelantar el entrenamiento o la evaluacion se instancia a tf.train.Saver.

saver = tf.train.Saver()

En el loop de entrenamiento la funcion saver.save() periodicamente se llamara para escribir un archivo checkpoint a  la carpeta de entrenamiento con los valores actuales de las variables de entrenamiento.

En algun punto del futuro el entrenamiento podria ser resumido usando la funcion saver.restore() y recargar los parametros del modelo.

saver.restore(sess, FLAGS.train_dir)

#Evaluando el Modelo

Cada mil pasos el codigo intentara evaluar el modelo contra el entrenamiento y los test de los datasets. La funcion do_eval() es llamada tres veces para el entrenamiento, validacion y los test de los datasets.

print 'Training Data Eval:'
do_eval(sess,
        eval_correct,
        images_placeholder,
        labels_placeholder,
        data_sets.train)
print 'Validation Data Eval:'
do_eval(sess,
        eval_correct,
        images_placeholder,
        labels_placeholder,
        data_sets.validation)
print 'Test Data Eval:'
do_eval(sess,
        eval_correct,
        images_placeholder,
        labels_placeholder,
        data_sets.test)

#Construir el Grafo Eval

Antes de abrir el grafo por default los datos para test deben extraerse llamando a la funcion get_data(train=False)  con el conjunto de parametros para agarrar el test dataset.

test_all_images, test_all_labels = get_data(train=False)

Antes de entrar al loop de entrenamiento la op eval debe haber construido la funcion evaluation() de mnist.py con los mismos parametros de logits/labels como la funcion loss().

eval_correct = mnist.evaluation(logits, labels_placeholder)

La funcion de evaluacion simplemente genera una op tf.nn.in_top_k  que automaticamente marca cada salida del modelo si la verdadera etiqueta se puede encontrar en las predicciones K most-likely. En este caso se establece un valor de k a 1 para solo considerar una prediccion si es correcta para la etiqueta.

eval_correct = tf.nn.in_top_k(logits, labels, 1)

#Salida Eval

Entonces se puede crear un loop pra llenar un feed_dict y llamando a sess.run() contra la op eval_correct a evaluar el modelo sobre el dataset dado.

for step in xrange(steps_per_epoch):

    feed_dict = fill_feed_dict(data_set images_placeholder, labels_placeholder)
    
    true_count += sess.run(eval_correct, feed_dict=feed_dict)
    
La variable true_count simplemente acumula todas las predicciones que la op in_top_k tiene determinadas a ser correctas. De ahi la precision puede ser calculada simplemente dvidiendo el numero total de ejemplos.

precision = float(true_count) / float(num_examples)
print '  Num examples: %d  Num correct: %d  Precision @ 1: %0.02f' % (num_examples, true_count, precision)